<a href="https://colab.research.google.com/github/Valancinis/WranglingData/blob/main/OORWrangle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import requests, io, pandas as pd
from datetime import datetime, timedelta
from google.colab import drive

  #reachout out to github repository
github_session = requests.Session()

    # providing raw url to download csv/excel from github
csv_url = 'https://raw.githubusercontent.com/Valancinis/WranglingData/main/OOR for Thermo Fisher Scientific - LT.csv'
excel_url = 'https://raw.githubusercontent.com/Valancinis/WranglingData/main/NavPOtest.xlsx'

download1 = github_session.get(csv_url).content
download2 = github_session.get(excel_url).content
importdataOOR = pd.read_csv(io.StringIO(download1.decode('utf-8')), on_bad_lines='skip', sep = ";", keep_default_na=False)
importdataOPO = pd.read_excel(download2)

   #picking needed data
oordf = importdataOOR.iloc[:,[2,10,16]].copy()
navdf = importdataOPO.iloc[:,[0,1,3]].copy()

oordf.rename({"Material": "ID", "Current ship date": "Delivery date"}, axis='columns', inplace=True)

   #setting 'Delivery date' column datatype to datetime
oordf['Delivery date'] = pd.to_datetime(oordf['Delivery date'], errors='coerce')
navdf['EDD'] = pd.to_datetime(navdf['EDD'], errors='coerce')

for i, row in oordf.iterrows():
  oordf.at[i,'Delivery date'] = (oordf.at[i,'Delivery date'] + timedelta(days=8))

navdf['Ddelta'] = navdf['EDD'] - oordf['Delivery date']

  #merge 2 dataframes
mergeddf = pd.merge(oordf, navdf)
  #Add a new column
mergeddf['Update'] = ""

  #removing non needed column
del mergeddf['EDD']

  #looping through the dataframe, turning timedelta objects to int and updating "Update" column values if row needs to be updated in the ERP system
for i in mergeddf.index:
  mergeddf.loc[i, 'Ddelta'] = int(mergeddf.loc[i,'Ddelta']/timedelta(days = 1))
  n = mergeddf.loc[i, 'Ddelta']
  if (n > 9) or (n < -9):
    mergeddf.loc[i,'Update'] = "True"
  else:
    mergeddf.loc[i,'Update'] = "False"

print(oordf.head(), '\n' ,navdf.head(), '\n', mergeddf.head())

   #Exporting csv to google drive
   #Mounting drive. Need to do this once
def mountingDrive():
    drive.mount('/content/drive')

  #Exporting
def exportingOor():
   #where to save path
  path = '/content/drive/My Drive/Python/output.csv'
  with open(path, 'w', encoding = 'utf-8-sig') as f:
    oordf.to_csv(f)
    print("File exported")

  #Exporting
def exportingOpenPOs():
  #where to save path
  path = '/content/drive/My Drive/Python/openpos.csv'
  with open(path, 'w', encoding = 'utf-8-sig') as f:
    navdf.to_csv(f)
    print("File exported")

      #Exporting
def exportingfinal():
  #where to save path
  #path = '/content/drive/My Drive/Python/final.csv'
  #with open(path, 'w') as f:
    #mergeddf.to_excel(f)
  mergeddf.to_excel(r'/content/drive/My Drive/Python/final.xlsx', index = False)
  print("File exported")

#mountingDrive()
  #Exporting files
#exportingOor()
#exportingOpenPOs()
exportingfinal()

            PO           ID Delivery date
0  PO-20-12297      916-021    2022-05-26
1  PO-20-13087    5000-1012    2022-06-23
2  PO-21-02186  342020-0125    2022-06-02
3  PO-21-03547      AB-2005    2022-06-16
4  PO-21-04128       3465NK    2022-09-08 
             PO           ID        EDD   Ddelta
0  PO-20-12297      916-021 2022-05-26   0 days
1  PO-20-13087    5000-1012 2022-08-17  55 days
2  PO-21-02186  342020-0125 2022-10-12 132 days
3  PO-21-03547      AB-2005 2022-12-02 169 days
4  PO-21-04128       3465NK 2022-08-31  -8 days 
             PO           ID Delivery date Ddelta Update
0  PO-20-12297      916-021    2022-05-26      0  False
1  PO-20-13087    5000-1012    2022-06-23     55   True
2  PO-21-02186  342020-0125    2022-06-02    132   True
3  PO-21-03547      AB-2005    2022-06-16    169   True
4  PO-21-04128       3465NK    2022-09-08     -8  False
File exported
